# 참고 코드
- private 2위 AutoML을 이용한 앙상블(https://www.dacon.io/competitions/official/236038/codeshare/7283?page=2&dtype=recent)
- 해당 코드를 참조하게 된 이유 : Dacon에서 AutoML을 사용하면 더 결과가 좋게 나온다는 이야기를 보았기 때문임. 타 모델에 비해서 얼마나 좋은지 확인하고 싶었음.

## Auto ML이란
- 자동화 된 기계 학습은 기계 학습을 실제 문제에 적용하는 프로세스를 자동화하는 프로세스
- pycaret 에서도 Auto ML 존재
- 난 supervised.automl을 사용했고, pycaret을 사용해서 다시 진행해보고 싶음

# import

In [4]:
#anaconda prompt 관리자 권한으로 실행 후
#conda install -c conda-forge mljar-supervised
#해당 코드 실행시켜서 설치함.

In [17]:
import pandas as pd
import numpy as np
import random
import os

import warnings
warnings.filterwarnings('ignore') 

from supervised.automl import AutoML

# 랜덤 시드와 폰트 설정

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [6]:
import matplotlib
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
    
matplotlib.rcParams['axes.unicode_minus'] = False  

# 데이터 로드 및 feature Engineering

In [7]:
train_df = pd.read_csv("train.csv")
test_df =  pd.read_csv("test.csv")

In [8]:
train_df["엽록소mean"] = train_df.filter(regex="엽록소").mean(axis=1)
train_df["엽록소std"]  = train_df.filter(regex="엽록소").std(axis=1)
train_df["엽록소diff"] = train_df["2022-11-28 엽록소"] - train_df["2022-09-01 엽록소"]

train_df["새순mean"]   = train_df.filter(regex="새순").mean(axis=1)
train_df["새순std"]    = train_df.filter(regex="새순").std(axis=1)
train_df["새순diff"] = train_df["2022-11-28 새순"] - train_df["2022-09-01 새순"]

test_df["엽록소mean"] = test_df.filter(regex="엽록소").mean(axis=1)
test_df["엽록소std"]  = test_df.filter(regex="엽록소").std(axis=1)
test_df["엽록소diff"] = test_df["2022-11-28 엽록소"] - test_df["2022-09-01 엽록소"]

test_df["새순mean"]   = test_df.filter(regex="새순").mean(axis=1)
test_df["새순std"]    = test_df.filter(regex="새순").std(axis=1)
test_df["새순diff"]   = test_df["2022-11-28 새순"] - test_df["2022-09-01 새순"]

- 날짜 데이터가 포함되어 있어 그에 따른 feature를 넣고자 함
    - 엽록소와 새순은 평균과 std 추가
    - 첫 날짜와 마지막 날짜의 차 추가 : diff

# EDA

In [ ]:
train_df.iloc[:, 1:7].describe()

In [ ]:
test_df.iloc[:, 1:6].describe()

In [ ]:
train_df.iloc[:, 1:].isna().sum().sum() 
#sum()을 두번 씀으로써 결측치 총합을 확인할 수 있음

In [ ]:
train_df.iloc[:, 1:7].corr()
#상관관계 확인

- 데이터 분포에는 큰 차이가 없고, 결측치 없음
- 새순과의 높은 상관관계 파악 가능

In [ ]:
sns.scatterplot(data = train.df.filter(regex = '엽록소').iloc[13].values)

In [ ]:
sns.scatterplot(data = train.df.filter(regex = '새순').iloc[9].values)

- 엽록소와는 큰 관계 없음
- 9월과 첫 날이 가장 중요
- 엽록소와 새순은 날이 갈수록 줄어드는 패턴

In [ ]:
sns.scatter(data = train_df[['착과량(int)', '2022-09-01 새순']], 
            x = '착과량(int)', y = '2022-09-01 새순')

- 그룹지어 나타나는 부분 존재
- 새순으로 대략 추정이 가능하나, 구분 가능한 feature가 필요해보임
    - 5의 새순은 착과량 0~100 사이의 값을 가질 수 있음. 
    - 더 이상 나눌 수 없다면, 5의 새순을 가진 데이터들의 착과량의 mean value를 가지게 될 것.
    - 2위분은 경계의 구분을 보여주는 feature를 찾지 못했음
        - ~4주차별 평균
        - 9월 평균
        - 9/1과 7, 14, 21, 28일과의 차
        - log, sqrt 등의 trainsform
     - 새순 이외 데이터/feature를 찾는다면 더 높은 성능의 추정이 가능할 것(현 대회에선 불가)

# 독립변수와 종속변수 분리

In [9]:
X = train_df.iloc[:, 2:]
y = train_df.iloc[:, 1]

In [10]:
automl = AutoML(mode="Compete", eval_metric='mae')

## AutoML의 매개변수
- AutoML은 다양한 모델로 훈련시키고 앙상블하게 함.

- mode :Explain, Perform, Compete가 기본 모드이며, AutoML constructor(생성자) 속 매개변수로 커스텀이 가능하다.
    - Explain : 사용자가 데이터를 설몇하고 이해하기 원할 때 사용
    - Perform : 실생활에 사용될 모델을 훈련시키길 원할 때 사용
    - Compete : 최대 성과를 낼 수 있게, 머신러닝 대회를 위해 사용.

- 기본 모드에서 적용되는 알고리즘 기법이 무엇인지 상세히 알고 싶거나(모드 마다 다름),커스텀 모드에 대해 이해하고 싶다면 링크 참조(https://supervised.mljar.com/features/modes/)
- eval_metric : 유효성 검사 데이터에 대한 평가 방식을 설정한다. 여기선 MAE(Mean Absolute Error, 평균절대오차) 사용하심.

In [11]:
automl.fit(X, y) # 시간 오래 걸림... 한시간 이상임...

AutoML directory: AutoML_1
The task is regression with evaluation metric mae
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mae 31.350505 trained in 1.22 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 4 models
1_DecisionTree mae 32.079328 trained in 7.7 seconds
2_DecisionTree mae 31.09015 trained in 8.51 seconds
3_DecisionTree mae 31.09015 trained in 7.98 seconds
4_Linear mae 

[146]	train's l1: 0.0428939	validation's l1: 0.14025
[147]	train's l1: 0.0425128	validation's l1: 0.140295
[148]	train's l1: 0.0421806	validation's l1: 0.140305
[149]	train's l1: 0.0418604	validation's l1: 0.140386
[150]	train's l1: 0.0414744	validation's l1: 0.140437
[151]	train's l1: 0.0411262	validation's l1: 0.140511
[152]	train's l1: 0.0407515	validation's l1: 0.140531
[153]	train's l1: 0.0404358	validation's l1: 0.140553
[154]	train's l1: 0.0400894	validation's l1: 0.1406
[155]	train's l1: 0.0398445	validation's l1: 0.140651
[156]	train's l1: 0.0395631	validation's l1: 0.140578
[157]	train's l1: 0.0392492	validation's l1: 0.140678
[158]	train's l1: 0.0389489	validation's l1: 0.14064
[159]	train's l1: 0.0386672	validation's l1: 0.140676
[160]	train's l1: 0.0383349	validation's l1: 0.140538
[161]	train's l1: 0.0380774	validation's l1: 0.140526
[162]	train's l1: 0.0378266	validation's l1: 0.140471
[163]	train's l1: 0.0375646	validation's l1: 0.140407
[164]	train's l1: 0.0372543	vali

[30]	train's l1: 0.208339	validation's l1: 0.212805
[31]	train's l1: 0.200038	validation's l1: 0.205564
[32]	train's l1: 0.192313	validation's l1: 0.19895
[33]	train's l1: 0.184973	validation's l1: 0.192967
[34]	train's l1: 0.178122	validation's l1: 0.18771
[35]	train's l1: 0.171569	validation's l1: 0.183196
[36]	train's l1: 0.165376	validation's l1: 0.178553
[37]	train's l1: 0.159628	validation's l1: 0.174228
[38]	train's l1: 0.154155	validation's l1: 0.170548
[39]	train's l1: 0.149116	validation's l1: 0.167182
[40]	train's l1: 0.144348	validation's l1: 0.163808
[41]	train's l1: 0.139984	validation's l1: 0.160943
[42]	train's l1: 0.135911	validation's l1: 0.158437
[43]	train's l1: 0.131934	validation's l1: 0.15626
[44]	train's l1: 0.128301	validation's l1: 0.15399
[45]	train's l1: 0.124981	validation's l1: 0.152215
[46]	train's l1: 0.12175	validation's l1: 0.150493
[47]	train's l1: 0.118852	validation's l1: 0.148618
[48]	train's l1: 0.116101	validation's l1: 0.147333
[49]	train's l1: 

[62]	train's l1: 0.0912322	validation's l1: 0.13474
[63]	train's l1: 0.0900998	validation's l1: 0.134408
[64]	train's l1: 0.0889365	validation's l1: 0.134002
[65]	train's l1: 0.0878109	validation's l1: 0.133952
[66]	train's l1: 0.086883	validation's l1: 0.133656
[67]	train's l1: 0.0858916	validation's l1: 0.133305
[68]	train's l1: 0.0849272	validation's l1: 0.133216
[69]	train's l1: 0.0840596	validation's l1: 0.133228
[70]	train's l1: 0.0832738	validation's l1: 0.133151
[71]	train's l1: 0.0823389	validation's l1: 0.133067
[72]	train's l1: 0.0814058	validation's l1: 0.132914
[73]	train's l1: 0.0805941	validation's l1: 0.132867
[74]	train's l1: 0.0797989	validation's l1: 0.132617
[75]	train's l1: 0.0790484	validation's l1: 0.132743
[76]	train's l1: 0.0782808	validation's l1: 0.132666
[77]	train's l1: 0.0775088	validation's l1: 0.132438
[78]	train's l1: 0.0767032	validation's l1: 0.132474
[79]	train's l1: 0.0759566	validation's l1: 0.132576
[80]	train's l1: 0.0752522	validation's l1: 0.13

[50]	train's l1: 0.109807	validation's l1: 0.171292
[51]	train's l1: 0.10754	validation's l1: 0.170415
[52]	train's l1: 0.105417	validation's l1: 0.169462
[53]	train's l1: 0.103385	validation's l1: 0.168628
[54]	train's l1: 0.101484	validation's l1: 0.167913
[55]	train's l1: 0.099647	validation's l1: 0.167319
[56]	train's l1: 0.0978972	validation's l1: 0.166668
[57]	train's l1: 0.0961856	validation's l1: 0.16598
[58]	train's l1: 0.0947078	validation's l1: 0.165384
[59]	train's l1: 0.0932059	validation's l1: 0.164764
[60]	train's l1: 0.0918618	validation's l1: 0.164318
[61]	train's l1: 0.0905951	validation's l1: 0.163664
[62]	train's l1: 0.089381	validation's l1: 0.163201
[63]	train's l1: 0.0881552	validation's l1: 0.162781
[64]	train's l1: 0.0870521	validation's l1: 0.16228
[65]	train's l1: 0.0859937	validation's l1: 0.161837
[66]	train's l1: 0.0849128	validation's l1: 0.16151
[67]	train's l1: 0.0839221	validation's l1: 0.161163
[68]	train's l1: 0.0829383	validation's l1: 0.160928
[69]

...

[91]	train's l1: 0.0446729	validation's l1: 0.126526
[92]	train's l1: 0.0441702	validation's l1: 0.12657
[93]	train's l1: 0.04369	validation's l1: 0.126435
[94]	train's l1: 0.0432754	validation's l1: 0.126606
[95]	train's l1: 0.0428641	validation's l1: 0.126547
[96]	train's l1: 0.0424206	validation's l1: 0.126495
[97]	train's l1: 0.0419702	validation's l1: 0.12655
[98]	train's l1: 0.0414994	validation's l1: 0.126614
[99]	train's l1: 0.0410646	validation's l1: 0.126471
[100]	train's l1: 0.0405774	validation's l1: 0.12644
[101]	train's l1: 0.040192	validation's l1: 0.126444
[102]	train's l1: 0.0398124	validation's l1: 0.126318
[1]	train's l1: 0.779702	validation's l1: 0.767552
[2]	train's l1: 0.702989	validation's l1: 0.69445
[3]	train's l1: 0.634212	validation's l1: 0.628689
[4]	train's l1: 0.572719	validation's l1: 0.569753
[5]	train's l1: 0.517621	validation's l1: 0.516688
[6]	train's l1: 0.468165	validation's l1: 0.47017
[7]	train's l1: 0.423835	validation's l1: 0.429007
[8]	train's 

[49]	train's l1: 0.0715063	validation's l1: 0.13706
[50]	train's l1: 0.0707922	validation's l1: 0.13746
[51]	train's l1: 0.0699327	validation's l1: 0.137619
[52]	train's l1: 0.0689756	validation's l1: 0.137424
[53]	train's l1: 0.0681236	validation's l1: 0.137406
[54]	train's l1: 0.0673044	validation's l1: 0.137598
[55]	train's l1: 0.0664307	validation's l1: 0.137536
[56]	train's l1: 0.0657319	validation's l1: 0.137545
[57]	train's l1: 0.064897	validation's l1: 0.137562
[58]	train's l1: 0.0641552	validation's l1: 0.13808
[59]	train's l1: 0.0634302	validation's l1: 0.138214
[60]	train's l1: 0.0626915	validation's l1: 0.13823
[61]	train's l1: 0.0620518	validation's l1: 0.138182
[62]	train's l1: 0.0613493	validation's l1: 0.138002
[63]	train's l1: 0.0605265	validation's l1: 0.138385
[64]	train's l1: 0.0598261	validation's l1: 0.138277
[65]	train's l1: 0.0591475	validation's l1: 0.138295
[66]	train's l1: 0.0584241	validation's l1: 0.138163
[67]	train's l1: 0.0577229	validation's l1: 0.13793

[59]	train's l1: 0.0625478	validation's l1: 0.133581
[60]	train's l1: 0.0618355	validation's l1: 0.133609
[61]	train's l1: 0.0610587	validation's l1: 0.133568
[62]	train's l1: 0.0602842	validation's l1: 0.133499
[63]	train's l1: 0.0595467	validation's l1: 0.13349
[64]	train's l1: 0.0588347	validation's l1: 0.133479
[65]	train's l1: 0.058159	validation's l1: 0.133637
[66]	train's l1: 0.0574748	validation's l1: 0.133857
[67]	train's l1: 0.0567628	validation's l1: 0.133709
[68]	train's l1: 0.0561518	validation's l1: 0.133626
[69]	train's l1: 0.0555669	validation's l1: 0.133547
[70]	train's l1: 0.0547899	validation's l1: 0.133511
[71]	train's l1: 0.0542582	validation's l1: 0.133602
[72]	train's l1: 0.0536878	validation's l1: 0.133879
[73]	train's l1: 0.0530445	validation's l1: 0.133763
[74]	train's l1: 0.0524411	validation's l1: 0.133802
[75]	train's l1: 0.0517478	validation's l1: 0.133801
[76]	train's l1: 0.0511979	validation's l1: 0.133754
[77]	train's l1: 0.0506612	validation's l1: 0.13

[136]	train's l1: 0.0276133	validation's l1: 0.134313
[137]	train's l1: 0.0273235	validation's l1: 0.134374
[1]	train's l1: 0.777398	validation's l1: 0.824031
[2]	train's l1: 0.701062	validation's l1: 0.743446
[3]	train's l1: 0.632899	validation's l1: 0.671607
[4]	train's l1: 0.571594	validation's l1: 0.606857
[5]	train's l1: 0.516688	validation's l1: 0.548787
[6]	train's l1: 0.46748	validation's l1: 0.497587
[7]	train's l1: 0.423398	validation's l1: 0.452036
[8]	train's l1: 0.383945	validation's l1: 0.409285
[9]	train's l1: 0.348555	validation's l1: 0.373008
[10]	train's l1: 0.317008	validation's l1: 0.339642
[11]	train's l1: 0.288702	validation's l1: 0.309399
[12]	train's l1: 0.263568	validation's l1: 0.283538
[13]	train's l1: 0.241242	validation's l1: 0.260006
[14]	train's l1: 0.221396	validation's l1: 0.239771
[15]	train's l1: 0.20381	validation's l1: 0.222095
[16]	train's l1: 0.188305	validation's l1: 0.206217
[17]	train's l1: 0.17482	validation's l1: 0.193783
[18]	train's l1: 0.1

AutoML(eval_metric='mae', mode='Compete')

In [12]:
pred = automl.predict(test_df.iloc[:, 1:])

In [13]:
submission = pd.read_csv("sample_submission.csv")
submission.iloc[:, 1] = pred
submission

,ID,착과량(int)
0,TEST_0000,248.775722
1,TEST_0001,750.637517
2,TEST_0002,154.671856
3,TEST_0003,450.760301
4,TEST_0004,689.146905
...,...,...
2203,TEST_2203,749.212631
2204,TEST_2204,344.223748
2205,TEST_2205,354.169094
2206,TEST_2206,252.278634


In [15]:
submission.to_csv("automl comepete1.csv", index=False)

# 평가산식 NMAE
- 이 부분을 개선하고 비교하고 싶음

In [16]:
import numpy as np

def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [ ]:
NMAE()

In [20]:
automl2 = AutoML(mode="Compete", eval_metric=NMAE)

In [21]:
automl2.fit(X, y)

ValueError: Metric NMAE is not allowed in ML task: regression.                 Use 'rmse', 'mse', 'mae', 'r2', 'mape', 'spearman', or 'pearson'

In [ ]:
#pred2 = automl2.predict(test_df.iloc[:, 1:])

In [ ]:
#submission2 = pd.read_csv("sample_submission.csv")
#submission2.iloc[:, 1] = pred2
#submission2

In [ ]:
#submission2.to_csv("automl comepete2.csv", index=False)

## 개선하고 싶은 점
- pycaret의 AutoML로 작성 및 비교
- NMAE를 적용하고 싶음
- AutoML에서 선택된 모델들을 하이퍼파라미터 튜닝 등으로 성능 강화하고 싶음
- 가능하다면, 직접 앙상블 시켜보고 싶음